# <span style="color: green;">Log Filtering Logic</span>

In [1]:
import os
import pandas as pd

def combine_and_filter_csv_files(directory, object_name_filter):
    combined_data = pd.DataFrame()

    # Iterate over all CSV files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            
            try:
                # Read the current CSV file into a DataFrame
                df = pd.read_csv(file_path)
                
                # Check if the required columns exist
                required_columns = ['Timestamp', 'Object Name', 'Event']
                missing_columns = [col for col in required_columns if col not in df.columns]
                if missing_columns:
                    # print(f"Skipping {filename}: Missing columns {missing_columns}")
                    continue
                
                # Select only the required columns
                df = df[required_columns]

                # Append to the combined DataFrame
                combined_data = pd.concat([combined_data, df], ignore_index=True)
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    
    # Filter rows by object name
    filtered_data = combined_data[combined_data['Object Name'].isin(object_name_filter)]

    # Convert 'Timestamp' to datetime and sort
    filtered_data['Timestamp'] = pd.to_datetime(filtered_data['Timestamp'], errors='coerce')
    filtered_data = filtered_data.dropna(subset=['Timestamp']).sort_values(by='Timestamp')
    
    return filtered_data


# <span style="color: green;">UML Code Generation Logic</span>

In [2]:
import re

def create_plantuml_sequence(logs):
    plantuml_code = '@startuml\n'
    
    # Add participants (Object Names)
    participants = set()
    for log in logs:
        parts = log.split(",")
        object_name = parts[1].strip()
        participants.add(object_name)
    
    # Add participants to the sequence diagram
    for participant in participants:
        plantuml_code += f'participant {participant}\n'
    
    # Store event mappings
    event_mapping = {}
    event_counter = 1
    
    # Add interactions based on events
    for log in logs:
        parts = log.split(",")
        timestamp = parts[0].strip()  # Extract timestamp
        object_name = parts[1].strip()
        event = parts[2].strip()
        
        # Add event to the mapping with timestamp and event
        event_mapping[event_counter] = f'[{timestamp[11:]}] {event}'  # Keep only time portion (hh:mm:ss)
        
        # Replace event text with number
        plantuml_code += f'{object_name} -> {object_name} : {event_counter}\n'
    
        event_counter += 1
        
    # Add the note with the event mappings (timestamp will be shown here)
    plantuml_code += 'note right of ' + list(participants)[0] + '\n'
    
    for num, msg in event_mapping.items():
        plantuml_code += f'  {num}: {msg}\n'
    
    plantuml_code += 'end note\n'
    plantuml_code += '@enduml\n'
    
    return plantuml_code


# <span style="color: green;">Extracting Certain Records Logic</span>

In [11]:
import csv

csv_file_path = 'CombinedLogs.csv'

def get_specific_line_numbers(csv_file_path, line_numbers_to_store):
    stored_lines = []
    
    # Open the CSV file
    with open(csv_file_path, mode='r') as file:
        reader = csv.reader(file)
        
        # Enumerate through the rows in the CSV file
        for line_number, row in enumerate(reader, start=1):
            # If the current line number is in the list of line numbers to store, save it
            if line_number in line_numbers_to_store:
                stored_lines.append(','.join(row))  # Convert the list row to a string
        
    return stored_lines


# <span style="color: green;">Worker Entering Zone Scenario</span>

## Denied Entry


In [4]:
directory = "..\\logs"  # Replace with the path to your CSV files
object_name_filter = ['RoofZone_GateNode', 
                      'RoofZone_GateNode_controller',
                      'RoofZone_GateNode_scanner',
                      'RoofZone_GateNode_relay',
                      'RoofZone_GateNode_motor',
                      'RoofZone_ControlNode',
                      'RoofZone_ControlNode_controller',
                      ]  # Replace with the object names to filter
result = combine_and_filter_csv_files(directory, object_name_filter)

# Save the DataFrame to a CSV file
result.to_csv('combinedLogs.csv', index=False)



C:\Users\omarf\AppData\Local\Temp\ipykernel_25628\3719143575.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Timestamp'] = pd.to_datetime(filtered_data['Timestamp'], errors='coerce')


In [5]:
# Example log data as an array of strings
logs = [
'2024-11-26 12:38:16.956494400,RoofZone_GateNode_controller,[SUCCESS] Received field [ID] from object [RoofZone_GateNode_scanner]. Value [Bruno]',
'2024-11-26 12:38:16.971265800,RoofZone_GateNode_controller,Asked Parent Node [RoofZone_GateNode] about ID [Bruno]''s permission',
'2024-11-26 12:38:17.091120900,RoofZone_GateNode,Asked Control Node [RoofZone_ControlNode] about ID [Bruno]''s permission',
'2024-11-26 12:38:17.108066700,RoofZone_ControlNode,Gate [RoofZone_GateNode] queried permission status for ID [Bruno]',
'2024-11-26 12:38:17.110043800,RoofZone_ControlNode_controller,Parent Node [RoofZone_ControlNode] asked if ID [Bruno] is permitted to enter zone. Permission [DENIED]',
'2024-11-26 12:38:17.111659900,RoofZone_ControlNode,Gate [RoofZone_GateNode] queried permission status for ID [Bruno]. Permission [DENIED]',
'2024-11-26 12:38:17.124630100,RoofZone_GateNode,ID [Bruno]''s permission: [DENIED]',
'2024-11-26 12:38:17.151296600,RoofZone_GateNode_controller,ID [Bruno]''s permission: [DENIED]',
'2024-11-26 12:38:17.286158800,RoofZone_GateNode_controller,[SUCCESS] Switched object [RoofZone_GateNode_relay] position [0] to [OFF]'
]

# Generate PlantUML code
plantuml_sequence = create_plantuml_sequence(logs)
print(plantuml_sequence)


@startuml
participant RoofZone_GateNode
participant RoofZone_GateNode_controller
participant RoofZone_ControlNode
participant RoofZone_ControlNode_controller
RoofZone_GateNode_controller -> RoofZone_GateNode_controller : 1
RoofZone_GateNode_controller -> RoofZone_GateNode_controller : 2
RoofZone_GateNode -> RoofZone_GateNode : 3
RoofZone_ControlNode -> RoofZone_ControlNode : 4
RoofZone_ControlNode_controller -> RoofZone_ControlNode_controller : 5
RoofZone_ControlNode -> RoofZone_ControlNode : 6
RoofZone_GateNode -> RoofZone_GateNode : 7
RoofZone_GateNode_controller -> RoofZone_GateNode_controller : 8
RoofZone_GateNode_controller -> RoofZone_GateNode_controller : 9
note right of RoofZone_GateNode
  1: [12:38:16.956494400] [SUCCESS] Received field [ID] from object [RoofZone_GateNode_scanner]. Value [Bruno]
  2: [12:38:16.971265800] Asked Parent Node [RoofZone_GateNode] about ID [Bruno]s permission
  3: [12:38:17.091120900] Asked Control Node [RoofZone_ControlNode] about ID [Bruno]s permi

## Allowed Entry

In [6]:
directory = "..\\logs"  # Replace with the path to your CSV files
object_name_filter = ['RoofZone_GateNode', 
                      'RoofZone_GateNode_controller',
                      'RoofZone_GateNode_scanner',
                      'RoofZone_GateNode_relay',
                      'RoofZone_GateNode_motor',
                      'RoofZone_ControlNode',
                      'RoofZone_ControlNode_controller',
                      ]  # Replace with the object names to filter
result = combine_and_filter_csv_files(directory, object_name_filter)

# Save the DataFrame to a CSV file
result.to_csv('combinedLogs.csv', index=False)



C:\Users\omarf\AppData\Local\Temp\ipykernel_25628\3719143575.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Timestamp'] = pd.to_datetime(filtered_data['Timestamp'], errors='coerce')


In [7]:
# Example log data as an array of strings
logs = [
'2024-11-26 12:38:21.887168400,RoofZone_GateNode_controller,[SUCCESS] Received field [ID] from object [RoofZone_GateNode_scanner]. Value [Omar]',
'2024-11-26 12:38:21.887168400,RoofZone_GateNode_controller,Asked Parent Node [RoofZone_GateNode] about ID [Omar]\'s permission',
'2024-11-26 12:38:21.888161000,RoofZone_GateNode,Asked Control Node [RoofZone_ControlNode] about ID [Omar]\'s permission',
'2024-11-26 12:38:21.903017800,RoofZone_ControlNode,Gate [RoofZone_GateNode] queried permission status for ID [Omar]',
'2024-11-26 12:38:21.903379700,RoofZone_ControlNode_controller,Parent Node [RoofZone_ControlNode] asked if ID [Omar] is permitted to enter zone. Permission [ALLOWED]',
'2024-11-26 12:38:21.903379700,RoofZone_ControlNode,Gate [RoofZone_GateNode] queried permission status for ID [Omar]. Permission [ALLOWED]',
'2024-11-26 12:38:21.918834800,RoofZone_GateNode,ID [Omar]\'s permission: [ALLOWED]',
'2024-11-26 12:38:21.918834800,RoofZone_GateNode_controller,ID [Omar]\'s permission: [ALLOWED]',
'2024-11-26 12:38:21.918834800,RoofZone_GateNode_controller,[SUCCESS] Switched object [RoofZone_GateNode_relay] position [0] to [ON]',

]

# Generate PlantUML code
plantuml_sequence = create_plantuml_sequence(logs)
print(plantuml_sequence)


@startuml
participant RoofZone_GateNode
participant RoofZone_GateNode_controller
participant RoofZone_ControlNode
participant RoofZone_ControlNode_controller
RoofZone_GateNode_controller -> RoofZone_GateNode_controller : 1
RoofZone_GateNode_controller -> RoofZone_GateNode_controller : 2
RoofZone_GateNode -> RoofZone_GateNode : 3
RoofZone_ControlNode -> RoofZone_ControlNode : 4
RoofZone_ControlNode_controller -> RoofZone_ControlNode_controller : 5
RoofZone_ControlNode -> RoofZone_ControlNode : 6
RoofZone_GateNode -> RoofZone_GateNode : 7
RoofZone_GateNode_controller -> RoofZone_GateNode_controller : 8
RoofZone_GateNode_controller -> RoofZone_GateNode_controller : 9
note right of RoofZone_GateNode
  1: [12:38:21.887168400] [SUCCESS] Received field [ID] from object [RoofZone_GateNode_scanner]. Value [Omar]
  2: [12:38:21.887168400] Asked Parent Node [RoofZone_GateNode] about ID [Omar]'s permission
  3: [12:38:21.888161000] Asked Control Node [RoofZone_ControlNode] about ID [Omar]'s permis

# <span style="color: green;">Safety Hook Scenario</span>

## Not Wearing

In [8]:
directory = "..\\logs"  # Replace with the path to your CSV files
object_name_filter = ['RoofZone_SmartRopeNode1', 
                      'RoofZone_SmartRopeNode1_controller', 
                      'RoofZone_ControlNode',
                      'RoofZone_ControlNode_controller',
                      'RoofZone_BuzzerNode',
                      'RoofZone_BuzzerNode_controller',
                      'RoofZone_SpeakerNode',
                      'RoofZone_SpeakerNode_controller'
                      ]  # Replace with the object names to filter
result = combine_and_filter_csv_files(directory, object_name_filter)

# Save the DataFrame to a CSV file
result.to_csv('combinedLogs.csv', index=False)



C:\Users\omarf\AppData\Local\Temp\ipykernel_25628\3719143575.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Timestamp'] = pd.to_datetime(filtered_data['Timestamp'], errors='coerce')


In [12]:

line_numbers_to_store = [221,223,225,229,233,251,253,254,255,256,258,260,269,270,272,271,276,275,278,277,279,280,282]
logs = get_specific_line_numbers(csv_file_path, line_numbers_to_store)

# Generate PlantUML code
plantuml_sequence = create_plantuml_sequence(logs)
print(plantuml_sequence)


@startuml
participant RoofZone_SpeakerNode_controller
participant RoofZone_BuzzerNode
participant RoofZone_BuzzerNode_controller
participant RoofZone_SmartRopeNode1
participant RoofZone_SmartRopeNode1_controller
participant RoofZone_ControlNode_controller
participant RoofZone_ControlNode
participant RoofZone_SpeakerNode
RoofZone_SmartRopeNode1_controller -> RoofZone_SmartRopeNode1_controller : 1
RoofZone_SmartRopeNode1_controller -> RoofZone_SmartRopeNode1_controller : 2
RoofZone_SmartRopeNode1 -> RoofZone_SmartRopeNode1 : 3
RoofZone_ControlNode -> RoofZone_ControlNode : 4
RoofZone_SmartRopeNode1 -> RoofZone_SmartRopeNode1 : 5
RoofZone_ControlNode_controller -> RoofZone_ControlNode_controller : 6
RoofZone_ControlNode -> RoofZone_ControlNode : 7
RoofZone_BuzzerNode -> RoofZone_BuzzerNode : 8
RoofZone_BuzzerNode -> RoofZone_BuzzerNode : 9
RoofZone_BuzzerNode_controller -> RoofZone_BuzzerNode_controller : 10
RoofZone_BuzzerNode -> RoofZone_BuzzerNode : 11
RoofZone_BuzzerNode -> RoofZone_B